# 5.3 Vertical and Horizontal Scaling with Examples


<div style="background-color: #f8f9fa; border: 1px solid #e9ecef; border-radius: 8px; padding: 10px; margin: 10px;">
<strong>📋 Workshop Contents</strong>
<ul style="line-height: 1.2;">
<li><a href="#Prerequisites">Prerequisites</a></li>
<li><a href="#Scaling-Strategies-in-Amazon-Aurora">Scaling Strategies in Amazon Aurora</a></li>
<li><a href="#1-Vertical-Scaling-Scaling-Up">1. Vertical Scaling (Scaling Up)</a></li>
<li><a href="#2-Horizontal-Scaling">2. Horizontal Scaling</a></li>
<li><a href="#3-Serverless-Scaling-Aurora-Serverless-v2">3. Serverless Scaling (Aurora Serverless v2)</a></li>
</ul>
</div>

In this module, we'll explore and implement multiple Aurora scaling strategies including vertical scaling (instance resizing), horizontal scaling with read replicas, serverless scaling with Aurora Serverless v2, and learn about advanced solutions like Aurora Limitless Database with practical Python code examples.

## Prerequisites
- Jupyter Notebook: You can launch a [free tier Amazon SageMaker Jupyter Notebook](../../1_Getting_Started_with_AWS/1.4_Setting_up_Your_Cookbook_Environment/README.MD)
- Access to an Amazon Aurora PostgreSQL database
- AWS CLI configured with appropriate permissions
- Jupyter Notebook environment with Python 3.6+ installed
- boto3 library (`pip install boto3`)
- psycopg2 library for PostgreSQL connections (`pip install psycopg2-binary`)
- Basic understanding of AWS RDS/Aurora concepts (completion of earlier modules is helpful but not mandatory)
- Basic knowledge of SQL and database performance concepts
- Make sure that you have required IAM permissions to connect to RDS/Aurora. If you need to update your IAM policy, see [Add RDS permissions to notebook](./5.3.3.1_Add-rds-permissions-to-notebook.md)

# Scaling Strategies in Amazon Aurora


## 1. Vertical Scaling (Scaling Up)

Vertical scaling increases the computational power of your existing database instance to handle growing workloads.

### Capabilities
- **Instance Range**: From entry-level (2 vCPU, 4GB RAM) to high-performance (128 vCPU, 1024GB RAM)
- **Transition Time**: 5-15 minutes with minimal application impact
- **Connection Handling**: Maintains existing connections during scaling
- **Automation**: Can be triggered by CloudWatch alarms based on performance metrics

### When to Use
- For workloads with increasing CPU or memory requirements
- When query performance degrades under load
- For temporary capacity needs during peak periods


### Example: Aurora Provisioned Vertical Scaling

Connect to AWS RDS using boto3, retrieve current instance information, modify the instance class to a larger size, and apply changes immediately with minimal downtime. Update the variables with your actual Aurora instance identifier, desired instance class (e.g., "db.r8g.4xlarge"), and AWS region before running.

In [ ]:
import boto3
import time
from IPython.display import display, HTML
import pandas as pd

def scale_aurora_instance(db_instance_identifier, new_instance_class, region='us-east-1'):
    """
    Scale Aurora provisioned instance to a new instance class
    
    Args:
        db_instance_identifier (str): Aurora instance identifier
        new_instance_class (str): New instance class (e.g., 'db.r8g.4xlarge')
        region (str): AWS region
    """
    rds = boto3.client('rds', region_name=region)
    
    try:
        # Get current instance info
        response = rds.describe_db_instances(DBInstanceIdentifier=db_instance_identifier)
        current_class = response['DBInstances'][0]['DBInstanceClass']
        
        print(f"Scaling instance: {db_instance_identifier}")
        print(f"From: {current_class} → To: {new_instance_class}")
        
        # Modify the instance
        rds.modify_db_instance(
            DBInstanceIdentifier=db_instance_identifier,
            DBInstanceClass=new_instance_class,
            ApplyImmediately=True
        )
        
        # Add tags to the instance
        rds.add_tags_to_resource(
            ResourceName=f"arn:aws:rds:{region}:*:db:{db_instance_identifier}",
            Tags=[
                {
                    'Key': 'CreationSource',
                    'Value': 'aws-database-cookbook-v2025.8'
                }
            ]
        )
        
        print("✅ Scaling operation initiated successfully!")
        print("⏳ This will take 5-15 minutes to complete...")
        
        return True
        
    except Exception as e:
        print(f"❌ Error scaling instance: {str(e)}")
        return False

# Example: Change the instance name and region
instance_id = "your-aurora-instance-name"
aws_region = "us-east-1"

# Execute the scaling, below example is scaling to r7g.4xlarge
scale_aurora_instance(
    db_instance_identifier=instance_id,
    new_instance_class="db.r7g.4xlarge",  # Change to the instance class that you would want to scale up
    region=aws_region
)


## 2. Horizontal Scaling
Horizontal scaling (also known as scaling out) distributes database workload across multiple instances to improve performance and availability.

### Aurora Horizontal Scaling Capabilities


| Read Replica Support | High Availability | Global Database Features | Load Balancing |
|---------------------|-------------------|--------------------------|---------------|
| • Up to 15 replicas per cluster<br>• Cross-AZ/region deployment<br>• Automatic failover promotion<br>• Reader endpoint load balancing | • 30-second failover time<br>• 6 copies across 3 AZs<br>• Continuous S3 backup<br>• Point-in-time recovery | • <1 second replication lag<br>• Up to 5 secondary regions<br>• ~1 minute RTO<br>• Regional read isolation | • Automatic connection routing<br>• Custom endpoint support<br>• DNS-level load balancing<br>• RDS Proxy integration |




Note: For a detailed implementation of horizontal scaling with read replicas, refer to [Section 5.2: Implementing Read Replicas](../5.2_Implementing_Read_Replicas/README.md).

## 3. Serverless Scaling (Aurora Serverless v2)

Aurora Serverless v2 provides on-demand, automatic scaling of database resources without requiring capacity planning. It continuously monitors workload and adjusts capacity in fine-grained increments within seconds, scaling from 0 to 128 ACUs (Aurora Capacity Units). This approach delivers the performance of provisioned instances with the cost-efficiency of pay-per-use pricing.

### Key Benefits

| Automatic Scaling | Cost Optimization | Operational Excellence |
|-------------------|-------------------|------------------------|
| • Sub-second response to workload changes<br>• Granular scaling in 0.5 ACU increments<br>• No disruption during scaling operations<br>• Predictive scaling based on workload patterns | • Pay-per-second billing model<br>• No minimum capacity commitment<br>• Automatic scale-down during idle periods<br>• Separate billing for compute and storage | • Zero maintenance window requirements<br>• Automated capacity management<br>• Built-in high availability<br>• Compatible with existing Aurora tools |

### Example: Aurora Serverless v2 Scaling Configuration

Connect to Aurora Serverless v2 cluster using boto3, retrieve current scaling configuration and cluster status, modify minimum and maximum ACU limits, and display configuration changes in a formatted table. Update the variables with your Aurora Serverless v2 cluster identifier, desired ACU limits (0.5 to 128), and AWS region before running.

In [ ]:
import boto3
import pandas as pd
from IPython.display import display, HTML

def update_aurora_serverless_scaling(cluster_identifier, min_acu=0.5, max_acu=16.0, region='us-east-1'):
    """
    Update Aurora Serverless v2 scaling configuration
    
    Args:
        cluster_identifier (str): Aurora cluster identifier
        min_acu (float): Minimum ACU (0.5 to 128)
        max_acu (float): Maximum ACU (0.5 to 128)
        region (str): AWS region
    
    Returns:
        dict: Response from AWS API
    """
    rds = boto3.client('rds', region_name=region)
    
    try:
        print(f"Updating scaling configuration for cluster: {cluster_identifier}")
        print(f"Min ACU: {min_acu}, Max ACU: {max_acu}")
        
        response = rds.modify_db_cluster(
            DBClusterIdentifier=cluster_identifier,
            ServerlessV2ScalingConfiguration={
                'MinCapacity': min_acu,
                'MaxCapacity': max_acu
            },
            ApplyImmediately=True
        )
        
        # Add tags to the cluster
        rds.add_tags_to_resource(
            ResourceName=f"arn:aws:rds:{region}:*:cluster:{cluster_identifier}",
            Tags=[
                {
                    'Key': 'CreationSource',
                    'Value': 'aws-database-cookbook-v2025.8'
                }
            ]
        )
        
        print("✅ Successfully updated scaling configuration!")
        return response
        
    except Exception as e:
        print(f"❌ Error updating scaling configuration: {str(e)}")
        return None

def get_cluster_info(cluster_identifier, region='us-east-1'):
    """
    Get current cluster scaling configuration
    """
    rds = boto3.client('rds', region_name=region)
    
    try:
        response = rds.describe_db_clusters(DBClusterIdentifier=cluster_identifier)
        cluster = response['DBClusters'][0]
        
        scaling_config = cluster.get('ServerlessV2ScalingConfiguration', {})
        
        info = {
            'Cluster ID': cluster['DBClusterIdentifier'],
            'Engine': cluster['Engine'],
            'Status': cluster['Status'],
            'Min ACU': scaling_config.get('MinCapacity', 'Not configured'),
            'Max ACU': scaling_config.get('MaxCapacity', 'Not configured')
        }
        
        df = pd.DataFrame([info])
        display(HTML(df.to_html(index=False)))
        
        return cluster
        
    except Exception as e:
        print(f"❌ Error getting cluster info: {str(e)}")
        return None

# Example usage
cluster_id = "your-aurora-cluster-name"
aws_region = "us-east-1"

# Get current configuration
print("Current Configuration:")
get_cluster_info(cluster_id, aws_region)

# Update scaling configuration
print("\nUpdating Configuration:")
result = update_aurora_serverless_scaling(
    cluster_identifier=cluster_id,
    min_acu=1.0,    # Set your desired minimum ACU
    max_acu=32.0,   # Set your desired maximum ACU
    region=aws_region
)

# Verify updated configuration
if result:
    print("\nUpdated Configuration:")
    get_cluster_info(cluster_id, aws_region)


## Additional Resources 📚

### Aurora Scaling Features
- [Aurora Scaling Documentation](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/Aurora.Managing.Performance.html)
- [Aurora Serverless v2](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/aurora-serverless-v2.html)
- [Aurora Read Replicas](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/Aurora.Replication.html)

### Performance & Monitoring
- [Aurora Performance Insights](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/USER_PerfInsights.html)
- [CloudWatch Metrics for Aurora](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/MonitoringAurora.html)
- [Aurora Auto Scaling](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/Aurora.Integrating.AutoScaling.html)

### Advanced Features
- [Aurora Global Database](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/aurora-global-database.html)
- [Aurora Limitless Database](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/limitless.html)
- [Aurora Best Practices](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/Aurora.BestPractices.html)